In [2]:
from sklearn.metrics import confusion_matrix
y_true = [1, 0, 1, 1, 0, 1]
y_pred = [0, 0, 1, 1, 0, 1]
confusion_matrix(y_true, y_pred) #실제값 행 번호, 예측값 열 번호

array([[2, 0],
       [1, 3]])

In [3]:
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
(tn, fp, fn, tp)

(2, 0, 1, 3)

In [4]:
import numpy as np
from sklearn.metrics import accuracy_score
y_pred = np.array([0, 1, 1, 0])
y_true = np.array([0, 1, 0, 0])
sum(y_true == y_pred) / len(y_true)

0.75

In [5]:
accuracy_score(y_true, y_pred) #함수 사용을 선호

0.75

In [6]:
import numpy as np
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
y_pred = np.array([0, 1, 1, 0, 1, 1, 1, 0])
y_true = np.array([0, 1, 0, 0, 0, 0, 1, 1])

In [7]:
precision_score(y_true, y_pred) #정밀도

0.4

In [8]:
recall_score(y_true, y_pred) # 민감도

0.6666666666666666

In [9]:
f1_score(y_true, y_pred) # F1 스코어

0.5

In [10]:
# 시그모이드 함수
def sigmoid(z):
    return 1 / (1 + np.exp(z))

In [ ]:
# 가설함수
def hypothesis_function(x, theta):
    z = (np.dot(-x,theta))
    return sigmoid(z)

In [ ]:
# 비용함수 (구현) - 이미 구현 되어 있음
def compute_cost(x, y, theta):
    m = y.shape[0]
    J = (-1.0 / m) * (
    y.T.dot(np.log(hypothesis_function(x,theta))) + \
    (1-y).T.dot(np.log(1- hypothesis_function(x,theta))))
    
    return J

In [ ]:
# 경사하강법(가중치 업데이트) - 오차를 최소화 하는 것을 목표로 한다.
def minimize_gradient(x, y, theta, iterations=100000, alpha=0.01):
    m = y.size
    cost_history = []
    theta_history = []
    
    for _ in range(iterations):
        original_theta = theta
        for i in range(theta.size):
            partial_marginal = x[:, i].reshape(x.shape[0], 1)
            delta = hypothesis_function(x, original_theta) - y
            grad_i = delta.T.dot(partial_marginal)
            theta[i] = theta[i] - (alpha * grad_i)
            
        if (_ % 100) == 0:
            theta_history.append(theta)
            cost_history.append(compute_cost(x, y, theta))
    return theta, np.array(cost_history),np.array(theta_history)

In [ ]:
#-------------------------실제로 로지스틱 회귀 구현-------------------------

In [11]:
# 1.데이터 셋 준비
import pandas as pd
data_url= "http://www-stat.wharton.upenn.edu/~waterman/DataSets/uva.txt"
df = pd.read_table(data_url)
df[:5]

,who,Newbie,Age,Gender,Household Income,Sexual Preference,Country,Education Attainment,Major Occupation,Marital Status,Years on Internet
0,id74364,0,54.0,Male,$50-74,Gay male,Ontario,Some College,Computer,Other,4-6 yr
1,id84505,0,39.0,Female,Over $100,Heterosexual,Sweden,Professional,Other,Other,1-3 yr
2,id84509,1,49.0,Female,$40-49,Heterosexual,Washington,Some College,Management,Other,Under 6 mo
3,id87028,1,22.0,Female,$40-49,Heterosexual,Florida,Some College,Computer,Married,6-12 mo
4,id76087,0,20.0,Male,$30-39,Bisexual,New Jersey,Some College,Education,Single,1-3 yr


In [12]:
# 2.데이터 전처리 - (필요없는 데이터 드롭)
df.pop('who') #who열 제거
df.pop('Country') #Country열 제거
df.pop('Years on Internet')#Years on Internet열 제거

df.dtypes

Newbie                    int64
Age                     float64
Gender                   object
Household Income         object
Sexual Preference        object
Education Attainment     object
Major Occupation         object
Marital Status           object
dtype: object

In [13]:
# 데이터 타입 변환
category_cols = ['Gender'
    , 'Household Income',
    'Sexual Preference',
    'Education Attainment',
    'Major Occupation', "Marital Status"]

for col in category_cols:
    df[col] = df[col].astype('category') # 범주형 데이터 타입 -> 카테고리 타입으로 변환 (의미 명확성, 처리 용이)
    
df.dtypes

Newbie                     int64
Age                      float64
Gender                  category
Household Income        category
Sexual Preference       category
Education Attainment    category
Major Occupation        category
Marital Status          category
dtype: object

In [14]:
# 결측값 확인 및 채우기
df_onehot = pd.get_dummies(df) #category타입만 변환 - 원핫 인코딩 
df_onehot.shape  # 차원 확인 (행,열)

(19583, 38)

In [15]:
# isnull() 함수는 데이터프레임의 각 요소가 결측치인지 아닌지를 boolean 형태로 반환하고,
# sum() 함수는 각 열별로 True (결측치)의 개수를 합산하여 결측치의 총 개수를 계산한다. 
# 이 결과를 통해 각 변수에서 결측치의 수를 파악할 수 있다. 즉, 결측치 확인
df_onehot.isnull().sum()   

Newbie                                 0
Age                                  561
Gender_Female                          0
Gender_Male                            0
Household Income_$10-19                0
Household Income_$20-29                0
Household Income_$30-39                0
Household Income_$40-49                0
Household Income_$50-74                0
Household Income_$75-99                0
Household Income_Over $100             0
Household Income_Under $10             0
Sexual Preference_Bisexual             0
Sexual Preference_Gay male             0
Sexual Preference_Heterosexual         0
Sexual Preference_Lesbian              0
Sexual Preference_Transgender          0
Sexual Preference_na                   0
Education Attainment_College           0
Education Attainment_Doctoral          0
Education Attainment_Grammar           0
Education Attainment_High School       0
Education Attainment_Masters           0
Education Attainment_Other             0
Education Attain

In [17]:
#Age 열의 결측치를 해당 열의 평균값으로 대체 
# pd.isnull(df_onehot['Age'])는 Age 열에서 결측치가 있는 위치를 boolean 배열로 반환하고, 
# loc[] 함수를 사용해 해당 위치를 선택한 후, df_onehot['Age'].mean()으로 Age 열의 평균 값을 계산하여 결측치를 이 값으로 채움 즉, 결측치 대체
df_onehot.loc[pd.isnull(df_onehot['Age']), "Age"] = df_onehot['Age'].mean()

In [18]:
# 3. 데이터 분리
x_data = df_onehot.iloc[:, 1:].values  # iloc[:, 1:]는 모든 행과 첫 번째 열을 제외한 모든 열을 선택
y_data = df_onehot.iloc[:, 0].values.reshape(-1, 1)  # 첫 번째 열을 종속 변수(y)로 설정 / reshape(-1, 1)은 y 데이터를 열 벡터 형태로 재구성하여 모델에 적합하게 함
y_data.shape, x_data.shape

((19583, 1), (19583, 37))

In [20]:
# MinMaxScaler는 특성의 스케일을 조정하여 데이터의 범위를 [0, 1]로 변환한다. 이는 각 특성의 최소값을 0, 최대값을 1로 조정하는 방식
from sklearn import preprocessing # Min-Max Standardzation 
min_max_scaler = preprocessing.MinMaxScaler()
x_data = min_max_scaler.fit_transform(x_data)

#train_test_split 함수: 이 함수는 데이터를 훈련 세트와 테스트 세트로 분리한다. test_size=0.33은 전체 데이터의 33%를 테스트 세트로 사용하고 나머지를 훈련 세트로 사용한다는 것을 의미
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.33, random_state=42) # 42 = 데이터를 분리할 때 일관된 방식
X_train.shape, X_test.shape

((13120, 37), (6463, 37))

In [21]:
# 4.로지스틱 회귀 학습
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(fit_intercept=True)   # 사이킷런(scikit-learn) 라이브러리의 로지스틱 회귀 모델 구현 /  True - 절편 항이 모델에 포함
logreg.fit(X_train, y_train.flatten())  # 실제 학습하는 부분

LogisticRegression()

In [22]:
# 5.값 예측하기와 성능 측정하기
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

y_true = y_test.copy()  # 예측 수행 (훈련된 로지스틱 회귀 모델을 사용하여 테스트 데이터 X_test에 대한 예측을 수행)
y_pred = logreg.predict(X_test)  # 혼동 행렬
#         예측값
#         0   1
#   실제값 0 TN  FP
#        1 FN  TP

confusion_matrix(y_true, y_pred)

array([[4487,  275],
       [1350,  351]])

In [23]:
accuracy_score(y_true, y_pred) # 정확도
precision_score(y_true, y_pred) # 정밀도
recall_score(y_true, y_pred) # 민감도
f1_score(y_true, y_pred) # F1 스코어

0.3016759776536313